In [12]:
! pip install jsonlines
import json
import jsonlines
import re
import numpy as np
import pandas as pd

# Данные

In [ ]:
! wget https://dl.fbaipublicfiles.com/LAMA/data.zip
! unzip data.zip
! wget https://dl.fbaipublicfiles.com/MLQA/MLQA_V1.zip
! unzip /content/MLQA_V1.zip

! wget http://files.deeppavlov.ai/datasets/sber_squad-v1.1.tar.gz
! tar xf /content/sber_squad-v1.1.tar.gz

--2020-07-30 12:07:51--  https://dl.fbaipublicfiles.com/MLQA/MLQA_V1.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75719050 (72M) [application/zip]
Saving to: ‘MLQA_V1.zip’

MLQA_V1.zip         100%[===================>]  72.21M  20.5MB/s    in 3.9s    

2020-07-30 12:07:55 (18.7 MB/s) - ‘MLQA_V1.zip’ saved [75719050/75719050]

Archive:  /content/MLQA_V1.zip
   creating: MLQA_V1/
   creating: MLQA_V1/dev/
  inflating: MLQA_V1/dev/dev-context-ar-question-ar.json  
  inflating: MLQA_V1/dev/dev-context-ar-question-de.json  
  inflating: MLQA_V1/dev/dev-context-ar-question-en.json  
  inflating: MLQA_V1/dev/dev-context-ar-question-es.json  
  inflating: MLQA_V1/dev/dev-context-ar-question-hi.json  
  inflating: MLQA_V1/dev/dev-context-ar-question-vi.json  
  inflating: MLQA_V1/d

Нужный формат

In [ ]:
num_examples = 10
with open('/content/data/Squad/test.jsonl') as f:
  for line in f.readlines()[:num_examples]:
    lama = json.loads(line)
    print(lama)

{'masked_sentences': ['To emphasize the 50th anniversary of the Super Bowl the [MASK] color was used.'], 'obj_label': 'gold', 'id': '56be4db0acb8001400a502f0_0', 'sub_label': 'Squad'}
{'masked_sentences': ['Von Miller plays in [MASK] position for the Denver Broncos.'], 'obj_label': 'linebacker', 'id': '56bf17653aeaaa14008c9513_0', 'sub_label': 'Squad'}
{'masked_sentences': ['The TGIF comedy Family Matters for the 1997-98 season was originally aired by [MASK].'], 'obj_label': 'CBS', 'id': '57273b69dd62a815002e99d8_0', 'sub_label': 'Squad'}
{'masked_sentences': ['Thomas Davis broke his [MASK] during the NFC Championship Game.'], 'obj_label': 'arm', 'id': '56bf3a223aeaaa14008c9576_0', 'sub_label': 'Squad'}
{'masked_sentences': ['The first choice in the NFL draft of 1998 was [MASK].'], 'obj_label': 'Manning', 'id': '56bf41013aeaaa14008c959b_0', 'sub_label': 'Squad'}
{'masked_sentences': ['Newton played as [MASK] during Super Bowl 50.'], 'obj_label': 'quarterback', 'id': '56bf41013aeaaa1400

Отбор нужных предложений: берем только с одним токеном в ответе

In [ ]:
with open('/content/MLQA_V1/dev/dev-context-es-question-es.json') as f:
  espanol = json.loads(f.read())

with open('/content/MLQA_V1/test/test-context-es-question-es.json') as f:
  espanol_test = json.loads(f.read())

with open('/content/train-v1.1.json') as f:
    rus = json.loads(f.read())

with open('/content/dev-v1.1.json') as f:
    rus_dev = json.loads(f.read())

Русский язык

In [ ]:
good_questions_rus = []
for chunk in rus['data']:
  for sent in chunk['paragraphs']:
    if len(sent['qas'][0]['answers'][0]['text'].split(' ')) == 1:
      good_questions_rus.append(sent)
len(good_questions_rus)

6906

In [ ]:
for chunk in good_questions_rus[:10]:
    print("Question: {}\nAnswer: {}".format(chunk['qas'][0]['question'], chunk['qas'][0]['answers'][0]['text']))
    print(chunk['qas'][0]['id'])
    print()

Question: Какие свойства в общем способно изменять смарт-стекло?
Answer: оптические
34107

Question: В каком году Грибоедов написал водевиль Кто брат, кто сестра, или Обман за обманом ?
Answer: 1823
47731

Question: Кем подтверждаются права на дворянские титулы в Лихтенштейне?
Answer: Князем
59780

Question: С каким счётом закончилась первая официальная игра между сборными Германии и Швейцарии 5 апреля 1908 года?
Answer: 5:3
15788

Question: К какому году PDP-10 уже устарели?
Answer: 1980
16088

Question: Как, в соответствии с произношением гласного в корне слова dúona хлеб , называют носителей южножемайтских говоров?
Answer: dū́nininkai
82472

Question: Одно из вымерших семейств змей?
Answer: Madtsoiidae.
82246

Question: Как сбрасыают листву вечнозеленые деревья?
Answer: порциями
26678

Question: Какое это вещество структурно?
Answer: Аморфное
12358

Question: Что генерирует ядерный реактор?
Answer: тепло
29604



In [ ]:
good_questions_rus_dev = []
for chunk in rus_dev['data']:
  for sent in chunk['paragraphs']:
    if len(sent['qas'][0]['answers'][0]['text'].split(' ')) == 1:
      good_questions_rus_dev.append(sent)

In [ ]:
for chunk in good_questions_rus_dev[:10]:
    print("Question: {}\nAnswer: {}".format(chunk['qas'][0]['question'], chunk['qas'][0]['answers'][0]['text']))
    print(chunk['qas'][0]['id'])
    print()

Question: Чем заболел Байрон в Миссолонги?
Answer: лихорадкой
12786

Question: Как отводятся излишки тепла у млекопитающих?
Answer: потоотделением
39919

Question: Какими русскими купцами был накоплен значимый торговый опыт?
Answer: купцами-новгородцами
37134

Question: Какой язык стал церковным и государственным языком княжеств, в которые входили ставшие оседлыми влахи?
Answer: церковнославянский
41220

Question: Какие индикаторы - дополнительные графики, формируемые на основе пересчёта значений, содержащихся в базовом графике цены?
Answer: технические
36699

Question: Какие две торпеды были изготовлены в течение 1937—1938 гг. с электродвигателем мощностью 45 кВт?
Answer: ЭТ-45
72734

Question: Кто увидел вражеский аэроплан во время розведки?
Answer: лётчики
31929

Question: Каким уровнем генетического вклада можно оценить организм, потомство которого слабо и плохо размножаеться?
Answer: низкими
54643

Question: Когда будет известна истинная оценка общего количества существующих видов

Далее отбор не зависимых от контеста предложений и составление из вопросительных утвердительных предложений (вручную)

Испанский: шаги такие же

In [ ]:
good_questions_espanol = []
for chunk in espanol['data']:
  for sent in chunk['paragraphs']:
    if len(sent['qas'][0]['answers'][0]['text'].split(' ')) == 1:
      good_questions_espanol.append(sent)

len(good_questions_espanol)

145

In [ ]:
for chunk in good_questions_espanol[:10]:
    print("Question: {}\nAnswer: {}".format(chunk['qas'][0]['question'], chunk['qas'][0]['answers'][0]['text']))
    print(chunk['qas'][0]['id'])
    print()

Question: ¿Cuántos días están bajo cero en invierno?
Answer: 40
4c5d3e150b460d8b01cab7fc43d8343b6d2df1be

Question: ¿En que alfabeto se escribía el kirguiso antes?
Answer: árabe
d6732b84ccc296fd04e20e4ac4a2a00817294b74

Question: ¿Para qué idioma se ofrecen servicios de interpretación?
Answer: kirguiso
e3ef1715e6a22099d2a795d2ff2fb68212552391

Question: ¿Cuál fue el nombre de su primer hijo?
Answer: Guillermo
08d5ad1cf72e3dd791ed788fe92cccb6d838fca2

Question: ¿En qué año ocupó Prusia Holstein?
Answer: 1866,
ab51230ce444b86e3421b79ac50a28a4320c1e71

Question: ¿Qué profesión tenía Gustav Freytag?
Answer: escritor
d78557a3727f5d8874b883b33e21581d562140fe

Question: ¿Cuándo se construyó el aeropuerto Poznan-Lawica Henryk Wieniawki?
Answer: 1913
72a7a8aeddcc1a17e039b288766ca3793c1db9f9

Question: ¿Con quién luchaba el jugador junto con su mayor enemigo?
Answer: TCTF
bf6e46e6e2f7b54c6e88819e976253236fa7db0c

Question: ¿En qué año nació Traoré?
Answer: 1974
ea7b1a28ca38687ee4f693aba6d8814361

In [ ]:
good_questions_espanol_test = []
for chunk in espanol_test['data']:
  for sent in chunk['paragraphs']:
    if len(sent['qas'][0]['answers'][0]['text'].split(' ')) == 1:
      good_questions_espanol_test.append(sent)
len(good_questions_espanol_test)

1548

In [ ]:
for chunk in good_questions_espanol[:10]:
    print("Question: {}\nAnswer: {}".format(chunk['qas'][0]['question'], chunk['qas'][0]['answers'][0]['text']))
    print(chunk['qas'][0]['id'])
    print()

Question: ¿Cuántos días están bajo cero en invierno?
Answer: 40
4c5d3e150b460d8b01cab7fc43d8343b6d2df1be

Question: ¿En que alfabeto se escribía el kirguiso antes?
Answer: árabe
d6732b84ccc296fd04e20e4ac4a2a00817294b74

Question: ¿Para qué idioma se ofrecen servicios de interpretación?
Answer: kirguiso
e3ef1715e6a22099d2a795d2ff2fb68212552391

Question: ¿Cuál fue el nombre de su primer hijo?
Answer: Guillermo
08d5ad1cf72e3dd791ed788fe92cccb6d838fca2

Question: ¿En qué año ocupó Prusia Holstein?
Answer: 1866,
ab51230ce444b86e3421b79ac50a28a4320c1e71

Question: ¿Qué profesión tenía Gustav Freytag?
Answer: escritor
d78557a3727f5d8874b883b33e21581d562140fe

Question: ¿Cuándo se construyó el aeropuerto Poznan-Lawica Henryk Wieniawki?
Answer: 1913
72a7a8aeddcc1a17e039b288766ca3793c1db9f9

Question: ¿Con quién luchaba el jugador junto con su mayor enemigo?
Answer: TCTF
bf6e46e6e2f7b54c6e88819e976253236fa7db0c

Question: ¿En qué año nació Traoré?
Answer: 1974
ea7b1a28ca38687ee4f693aba6d8814361

Данные были занесены в таблицу с колонками, соответствующим ключам из xLama (предложение, ответ, айди)

In [212]:
data_s = pd.read_excel("/content/Squad_rus.xls")
data_s.drop_duplicates("id", inplace=True)
data_s.obj_label = data_s.obj_label.astype(str)
len(data_s["id"])

202

In [213]:
Squad_rus = []
for index, data in data_s.iterrows():
  obj = {'masked_sentences':[data['sentences']], 'obj_label':data['obj_label'], 'id':data['id'], 'sub_label':"Squad"}
  Squad_rus.append(obj)

Примеры на русском

In [ ]:
for i in Squad_rus[:10]:
  print(i)

{'masked_sentences': ['Байрон заболел [MASK] в Миссолонги.'], 'obj_label': 'лихорадкой', 'id': 12786, 'sub_label': 'Squad'}
{'masked_sentences': ['Bзлишки тепла у млекопитающих отводятся [MASK].'], 'obj_label': 'потоотделением', 'id': 39919, 'sub_label': 'Squad'}
{'masked_sentences': ['Кристаллы различной формы имеют [MASK] вершины.'], 'obj_label': 'острые', 'id': 70862, 'sub_label': 'Squad'}
{'masked_sentences': ['Виноградов предложил [MASK] методов оценки модуля тригонометрических сумм.'], 'obj_label': 'два', 'id': 23098, 'sub_label': 'Squad'}
{'masked_sentences': ['Группы: Devo, Suicide, Television, Talking Heads являются представителями [MASK].'], 'obj_label': 'постпанка', 'id': 69511, 'sub_label': 'Squad'}
{'masked_sentences': ['Раковины [MASK] выставлялись на показ в интерьерах XIX века.'], 'obj_label': 'моллюсков', 'id': 33857, 'sub_label': 'Squad'}
{'masked_sentences': ['Круг кровообращения, функционирующий через легкие, называется [MASK].'], 'obj_label': 'малым', 'id': 19286, 

И теперь записываем в нужном формате

In [214]:
with jsonlines.open('test_russian.jsonl', mode='w') as writer:
    writer.write_all(Squad_rus)

Теперь то же самое для Испанского

In [208]:
data_s = pd.read_excel("/content/Spanish_SQUAD_data_prepared_sentences.xls")
data_s.drop_duplicates("id", inplace=True)
len(data_s["id"])

200

In [209]:
data_s.obj_label = data_s.obj_label.astype(str)

In [210]:
Squad_esp = []
for index, data in data_s.iterrows():
  obj = {'masked_sentences':[data['masked_sentences']], 'obj_label':data['obj_label'], 'id':data['id'], 'sub_label':"Squad"}
  Squad_esp.append(obj)

 Примеры

In [ ]:
for i in Squad_esp[:10]:
  print(i)

{'masked_sentences': ['Klaudia Gallovits-Hall es de nacionalidad  [MASK].'], 'obj_label': 'rumana', 'id': 'fc144486715b86fc8538e24b9dcf83aca08e3b36', 'sub_label': 'Squad'}
{'masked_sentences': ['Inglaterra puede separarse oficialmente en [MASK] areas.'], 'obj_label': 'nueve', 'id': '79335fe69a4757e3edf229b1ca6cc6e61ec7e428', 'sub_label': 'Squad'}
{'masked_sentences': ['La dinastía mogol se estableció en [MASK].'], 'obj_label': '1526', 'id': '179cfa1786b78adce6971d822061bfd7317d5fe0', 'sub_label': 'Squad'}
{'masked_sentences': ['El idioma número uno en términos de números de usuarios de Wikipedia es [MASK].'], 'obj_label': 'inglés', 'id': '370921c39d6ecf8b3dd662739e17d3646d89a36a', 'sub_label': 'Squad'}
{'masked_sentences': ['[MASK] teorizó sobre la cosmología en la que la Tierra tenía forma de un pilar.'], 'obj_label': 'Anaximandro', 'id': 'fe8e18a717d84104dce27dba6485db4bdd9e6f36', 'sub_label': 'Squad'}
{'masked_sentences': ['El álbum de estudio de Britney Spears que lanzó después del

И опять же запись в нужный формат

In [211]:
with jsonlines.open('test_espanol.jsonl', mode='w') as writer:
    writer.write_all(Squad_esp)

In [ ]:
with open('/content/MLQA_V1/dev/dev-context-en-question-en.json') as f:
  espanol = json.loads(f.read())

In [ ]:
good_questions_rus = []
for chunk in espanol['data']:
  for sent in chunk['paragraphs']:
    if len(sent['qas'][0]['answers'][0]['text'].split(' ')) == 1:
      good_questions_rus.append(sent)
len(good_questions_rus)

373

 # Часть 2 с переводом на испанский/немецким



С переводом наблюдается ряд проблем, так как нет однозначного соответствия: token_lang1 - token_lang2. Какие-то слова переводятся на другой язык целыми словосочетаниями или же наоборот; также в русском языке нет артиклей, характерных для испанского. Поэтому нет возможности запомнить айди токена, чтобыы заменить его потом на MASK. Да и в целом местами перевод все же не совсем адекватный и требует постобработки.

In [231]:
esp = []
with open('/content/test_espanol.jsonl') as f:
  for line in f.readlines():
    lama = json.loads(line)
    esp.append(lama)

rus = []
with open('/content/test_russian.jsonl') as f:
  for line in f.readlines():
    lama = json.loads(line)
    rus.append(lama)

In [ ]:
! pip install transformers
from transformers import MarianMTModel, MarianTokenizer
model_name = 'Helsinki-NLP/opus-mt-es-ru'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

Добавляем обратно слова вместо специального токена, чтобы получить более адекватный перевод

In [84]:
ids = []
triplets = []
for obj in rus:
  temp = obj['masked_sentences'][0].split(" ")
  for i, token in enumerate(obj['masked_sentences'][0].split(" ")):
      if token == "[MASK]":
        ids.append(i)
        temp[i] = obj['obj_label']
      if token == "[MASK].":
        ids.append(i)
        temp[i] = obj['obj_label']+"."
      if token == "[MASK],":
        ids.append(i)
        temp[i] = obj['obj_label']+","
        
  triplets.append([[" ".join(temp)], obj['obj_label'], obj['id']])

In [ ]:
num_ex = 10
for i in triplets[:num_ex]:
  print(i)

[['Байрон заболел лихорадкой в Миссолонги.'], 'лихорадкой', 12786]
[['Излишки тепла у млекопитающих отводятся потоотделением.'], 'потоотделением', 39919]
[['Кристаллы различной формы имеют острые вершины.'], 'острые', 70862]
[['Виноградов предложил два методов оценки модуля тригонометрических сумм.'], 'два', 23098]
[['Группы: Devo, Suicide, Television, Talking Heads являются представителями постпанка.'], 'постпанка', 69511]
[['Раковины моллюсков выставлялись на показ в интерьерах XIX века.'], 'моллюсков', 33857]
[['Круг кровообращения, функционирующий через легкие, называется малым.'], 'малым', 19286]
[['Водородные соединения других элементов являются газами.'], 'газами', 80930]
[['Жёлтая лихорадка передается через укус комара.'], 'Жёлтая', 49976]
[['Химическая реакция, происходящая при участии катализаторов, называется катализ.'], 'катализ', 7324]


Переводим и сохраняем

In [87]:
for i in triplets:
  translated = model.generate(**tokenizer.prepare_translation_batch(i[0]))
  tgt_text = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
  i[0] = tgt_text

data = pd.DataFrame(triplets)
data.to_csv('translated_to_rus')

Дальше опять дорабатываем вручную 

То же самое и для обратного перевода

In [114]:
ids = []
triplets = []
for obj in esp:
  temp = obj['masked_sentences'][0].split(" ")
  for i, token in enumerate(obj['masked_sentences'][0].split(" ")):
      if token == "[MASK]":
        ids.append(i)
        temp[i] = obj['obj_label']
      if token == "[MASK].":
        ids.append(i)
        temp[i] = obj['obj_label']+"."
      if token == "[MASK],":
        ids.append(i)
        temp[i] = obj['obj_label']+","
        
  triplets.append([[" ".join(temp)], obj['obj_label'], obj['id']])

In [116]:
num_ex = 10
for i in triplets[:num_ex]:
  print(i)

[['Klaudia Gallovits-Hall es de nacionalidad  rumana.'], 'rumana', 'fc144486715b86fc8538e24b9dcf83aca08e3b36']
[['Inglaterra puede separarse oficialmente en nueve areas.'], 'nueve', '79335fe69a4757e3edf229b1ca6cc6e61ec7e428']
[['La dinastía mogol se estableció en 1526.'], '1526', '179cfa1786b78adce6971d822061bfd7317d5fe0']
[['El idioma número uno en términos de números de usuarios de Wikipedia es inglés.'], 'inglés', '370921c39d6ecf8b3dd662739e17d3646d89a36a']
[['Anaximandro teorizó sobre la cosmología en la que la Tierra tenía forma de un pilar.'], 'Anaximandro', 'fe8e18a717d84104dce27dba6485db4bdd9e6f36']
[['El álbum de estudio de Britney Spears que lanzó después del segundo se llamó Britney.'], 'Britney', 'fb612186581e69a1cecafc6c10ab5ea282609597']
[['Piaget en su reloj en 1979 presentó el estilo vanguardista.'], 'vanguardista', 'b9c4f95cef0499bbc560673068ad3dcc90655835']
[['Se pasó a llamar Barclays en 1736.'], '1736', '0dba79afc1c60403b38c86282b1a02814b50625c']
[['Las polillas y l

In [117]:
for i in triplets:
  translated = model.generate(**tokenizer.prepare_translation_batch(i[0]))
  tgt_text = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
  i[0] = tgt_text

data = pd.DataFrame(triplets)
data.to_csv('translated_to_esp')

После приведения к нужному формату считываем полученные таблицы

In [201]:
data_esp_ru = pd.read_csv('/content/translated_to_esp_edited.csv', skipinitialspace=True)
data_esp_ru.drop(columns="Unnamed: 0", axis=1, inplace=True)
data_esp_ru.columns = ['masked_sentences', 'obj_label', 'id']
data_esp_ru.drop_duplicates('id', inplace=True)

In [205]:
Squad_esp_ru = []
for index, data in data_esp_ru.iterrows():
  obj = {'masked_sentences':[data['masked_sentences']], 'obj_label':data['obj_label'], 'id':data['id'], 'sub_label':"Squad"}
  Squad_esp_ru.append(obj)

и теперь для русско-испанского варианта то же самое

In [203]:
data_ru_esp = pd.read_csv('/content/translated_to_rus_edited.csv', skipinitialspace=True)
data_ru_esp.drop(columns="Unnamed: 0", axis=1, inplace=True)
data_ru_esp.columns = ['masked_sentences', 'obj_label', 'id']
data_ru_esp.drop_duplicates("id", inplace=True)

In [207]:
Squad_ru_esp = []
for index, data in data_ru_esp.iterrows():
  obj = {'masked_sentences':[data['masked_sentences']], 'obj_label':data['obj_label'], 'id':data['id'], 'sub_label':"Squad"}
  Squad_ru_esp.append(obj)

Последнее - пока надо соединить две части, в одно, чтобы получились одинаковые по содержанию датасеты на разных языках

In [235]:
test_russian_full = rus+Squad_esp_ru
test_espanol_full = Squad_ru_esp+esp

In [246]:
test_espanol_full[0], test_russian_full[0]

({'id': 12786,
  'masked_sentences': ["['Byron tuvo [MASK] en Missolong.']"],
  'obj_label': 'fiebre',
  'sub_label': 'Squad'},
 {'id': 12786,
  'masked_sentences': ['Байрон заболел [MASK] в Миссолонги.'],
  'obj_label': 'лихорадкой',
  'sub_label': 'Squad'})

In [250]:
test_espanol_full[60], test_russian_full[60]

({'id': 46253,
  'masked_sentences': ["['La definición clásica de probabilidad puede resumirse en caso de que haya un número [MASK] de valores posibles.']"],
  'obj_label': 'infinito',
  'sub_label': 'Squad'},
 {'id': 46253,
  'masked_sentences': ['Классическое определение вероятности можно обобщить на случай [MASK] количества возможных значений.'],
  'obj_label': 'бесконечного',
  'sub_label': 'Squad'})

In [247]:
with jsonlines.open('test_espanol_full.jsonl', mode='w') as writer:
    writer.write_all(test_espanol_full)

with jsonlines.open('test_russian_full.jsonl', mode='w') as writer:
    writer.write_all(test_russian_full)